# Eat Safe, Love

## Notebook Set Up

In [16]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [17]:
# Create an instance of MongoClient
mongo = MongoClient(port=27018)

In [18]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [19]:
# review the collections in our database
print(db.list_collection_names())

[]


In [20]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [15]:
# Find the establishments with a hygiene score of 20
query ={'scores.Hygiene':{'$eq': 20}}
results = establishments.find(query)
# Use count_documents to display the number of documents in the result

count_documents = establishments.count_documents(query)
# Display the first document in the results using pprint

print(f'There are {count_documents} number of documents in the result')

There are 0 number of documents in the result


In [ ]:
# Convert the result to a Pandas DataFrame
hygieneDf = pd.DataFrame(db.establishments.find(query))
# Display the number of rows in the DataFrame
print("There are " + str(len(hygieneDf['BusinessName']))+ rows")
# Display the first 10 rows of the DataFrame
hygieneDf.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [21]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"RatingValue":{"$gte":4}, "LocalAuthorityName":{'$regex':"London"}}

# Use count_documents to display the number of documents in the result
print(f'There are {db.establishments.count_documents(query)} establishments in London with a rating of 4+')

# Display the first document in the results using pprint
pprint(db.establishments.find_one(query))


There are 0 establishments in London with a rating of 4+
None


In [ ]:
# Convert the result to a Pandas DataFrame
LondonDf = pd.DataFrame(db.establishments.find(query))
# Display the number of rows in the DataFrame
print("There are " + str(len(LondonDf['BusinessName']))+ " rows")
# Display the first 10 rows of the DataFrame
LondonDf.head(10)


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5

penang_query = {"BusinessName":'Penang Flavours'}
penang = db.establishments.find(penang_query)
for i in penang:
    geo = i['geocode']
lat = geo['latitude']
long = geo['longitude']
degree_search = 0.01
query = {'geocode.latitude':{'$gte':lat-degree_search, '$lte':lat+degree_search},\
               'geocode.longitude':{'$gte':long-degree_search,'$lte':long+degree_search},\
         "RatingValue":5,# Rating value must equal 5
               }

query =
# Sort by hygiene score

sort = [('scores.Hygiene',1)]
limit = 5

results = db.establishments.find(query).sort(sort).limit(limit)
# Print the results
pprint(list(results))


In [ ]:
# Convert result to Pandas DataFrame
near_df = pd.DataFrame(db.establishments.find(query).sort(sort).limit(limit))
near_df

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
match_query = {'$match':{scores."Hygiene":0}}
# 2. Groups the matches by Local Authority
group_query = {'group':{'_id':'$LocalAuthorityName', 'count':{'sum':1}}}
# 3. Sorts the matches from highest to lowest
sort = {'$sort':{'count':-1}}
# Print the number of documents in the result
pipeline =[match_query,group_query,sort]
results = list(db.establishments.aggregate(pipeline))
pprint(len(results))
# Print the first 10 results
pprint(results[0:10])

In [ ]:
# Convert the result to a Pandas DataFrame
pipeDf = pd.DataFrame(db.establishments.aggregate(pipeline))
# Display the number of rows in the DataFrame
print("There are " + str(len(pipeDf["_id"])) + "documents that match the query")
# Display the first 10 rows of the DataFrame
pipeDf.head(10)